In [2]:
import openai
import json
import requests
import pprint
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import moviepy
from moviepy.editor import *

GPT_MODEL = "gpt-3.5-turbo-0613"

In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [4]:
def trim_video(file_name, starting_at, ending_at):
    """Remove n seconds from a file between two points"""

    video_clip = VideoFileClip(filename=file_name)
    audio_clip = video_clip.audio
    video_clip = video_clip.cutout(starting_at, ending_at)
    audio_clip = audio_clip.cutout(starting_at, ending_at)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile("outputfile.mp4", temp_audiofile='temp-audio.m4a', 
                               remove_temp=True, audio_codec='aac')




In [5]:
functions = [
    {
        "name": "trim_video",
        "description": "Trim seconds from a video",
        "parameters": {
            "type": "object",
            "properties": {
                "file_name": {
                    "type": "string",
                    "description": "The name and location of the video file to be trimmed",
                },
                 "starting_at": {
                    "type": "string",
                    "description": "The second at which to start trimming from video file e.g. 4",
                },
                   "ending_at": {
                    "type": "string",
                    "description": "The second at which to end the trimming from video file e.g. 12 seconds",
                },
            },
            "required": ["file_name, starting_at, ending_at"],
        },
    }
]


In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Can you trim a few seconds from a video for me?"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': 'Of course! Please provide me with the following details:\n1. The name and location of the video file you want to trim.\n2. The second at which you want to start trimming from the video file.\n3. The second at which you want to end the trimming from the video file.'}

In [7]:
messages.append({"role": "user", "content": "The video is called samplevideo.mp4 and i want to trim from second 4 to second 10"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'trim_video',
  'arguments': '{\n  "file_name": "samplevideo.mp4",\n  "starting_at": "4",\n  "ending_at": "10"\n}'}}

In [8]:
if assistant_message.get("function_call"):
    function_name = assistant_message["function_call"]["name"]
    function_args = json.loads(assistant_message["function_call"]["arguments"])

    # here we actually call the function
    function_response = trim_video(
        file_name=function_args.get("file_name"),
        ending_at=function_args.get("ending_at"),
        starting_at=function_args.get("starting_at")
    )

Moviepy - Building video outputfile.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video outputfile.mp4



Moviepy - Done !
Moviepy - video ready outputfile.mp4
